# 0.0 - Random Forest Regressor

Os parâmetros utilizados para treinar o algoritmo serão a profundidade máxima das árvores `max_depth` e o número de árvores na floresta, `n_estimators`. Os valores padrão de `max_depth` e `n_estimators` são `None` e `100`, respectivamente.  
Os detalhes dos outros parâmetros, bem como exemplos de uso do algoritmo, podem ser vistos na documentação oficial em: 

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

# 1.0 - Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

from itertools import product
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# 2.0 - Carregando os dados

In [2]:
# Treino
X_train = pd.read_csv('data/train/X_training.csv')
y_train = pd.read_csv('data/train/y_training.csv')

# Validação
X_val = pd.read_csv('data/validation/X_validation.csv')
y_val = pd.read_csv('data/validation/y_val.csv')

# Teste
X_test = pd.read_csv('data/test/X_test.csv')
y_test = pd.read_csv('data/test/y_test.csv')

# 3.0 - Funções

In [3]:
def metrics(y, y_pred, step, algorithm):
    """
    Calcula várias métricas de avaliação de um modelo de regressão e as retorna em um DataFrame.

    Parâmetros:
    -----------
    y : array-like
        Valores da variável resposta.
    
    y_pred : array-like
        Valores preditos pelo modelo.
    
    step : str
        Etapa do processo de modelagem (ex: 'treino', 'validação', 'teste').
    
    algorithm : str
        Nome do algoritmo usado.

    Retorno:
    --------
    df_metrics : pandas.DataFrame
        DataFrame contendo as métricas calculadas:
        - 'Algorithm': Nome do algoritmo.
        - 'Step': Etapa do processo (treino, validação ou teste).
        - 'R2': Coeficiente de determinação.
        - 'MSE': Erro quadrático médio.
        - 'RMSE': Raiz do erro quadrático médio.
        - 'MAE': Erro absoluto médio.
        - 'MAPE': Erro percentual absoluto médio.
    """
    # Calcula as métricas
    r2 = r2_score(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, y_pred)
    mape = mean_absolute_percentage_error(y, y_pred)

    # Cria o DataFrame com as métricas
    df_metrics = pd.DataFrame({   
        'Algorithm': [algorithm],
        'Step': [step],
        'R2': [np.round(r2, 4)],
        'MSE': [np.round(mse, 4)],
        'RMSE': [np.round(rmse, 4)],
        'MAE': [np.round(mae, 4)],
        'MAPE': [np.round(mape, 4)]
    })
    
    return df_metrics

def exporta_excel(nome_algoritmo):
    """
    Exporta os DataFrames de métricas para arquivos Excel em diferentes pastas.

    Args:
        nome_algoritmo (str): Nome do algoritmo utilizado.

    Returns:
        None
    """
    etapas = ['train', 'validation', 'test']
    df_list = [df_metrics_train, df_metrics_val, df_metrics_test]

    for etapa, df in zip(etapas, df_list):
        df.to_excel(f'metrics/{etapa}/{nome_algoritmo}_{etapa}.xlsx', index=False)

# 4.0 - Ajustando os dados

In [4]:
# transformação do formato DataFrame para Series, essencialmente um array unidimensional
y_train, y_val, y_test = [ df.loc[:, 'song_popularity'] for df in [y_train, y_val, y_test] ]

# 5.0 - Desempenho do modelo

## 5.1 - Dados de treino

In [5]:
# Definição do modelo
# Vamos inciar com 'max_depth' = None (valor padrão) e 'n_estimators' = 100 (valor padrão) 
random_forest_train = RandomForestRegressor( random_state = 0 )

# Treinamento do algoritmo. 
random_forest_train.fit(X_train, y_train)

# Predições sobre os dados de treino
y_pred_train = random_forest_train.predict(X_train)

# Métricas
df_metrics_train = metrics(y_train, y_pred_train, 'Train', 'Random Forest Regressor')
df_metrics_train

,Algorithm,Step,R2,MSE,RMSE,MAE,MAPE
0,Random Forest Regressor,Train,0.9028,46.4547,6.8158,4.8608,2.578


## 5.2 - Dados de validação

In [6]:
# Definição do modelo
random_forest_val = RandomForestRegressor( random_state = 0 )

# Treinamento do algoritmo. 
random_forest_val.fit(X_train, y_train)

# Predições sobre os dados de validação
y_pred_val = random_forest_val.predict(X_val)

# Métricas
df_metrics_val = metrics(y_val, y_pred_val, 'Validation', 'Random Forest Regressor')
df_metrics_val

,Algorithm,Step,R2,MSE,RMSE,MAE,MAPE
0,Random Forest Regressor,Validation,0.3351,317.4782,17.8179,13.0022,7.0309


## 5.3 - Ajuste fino dos hiperparâmetros (Etapa de *Fine Tuning*)

In [8]:
# array para os valores de 'max_depth' (máxima profundidade da arvore) a serem testados
max_depth_array = np.arange(2, 20, 1)

# lista para o número de estimadores (árvores da floresta)
n_estimators_list = [100, 500, 1000]

# Criar DataFrame para armazenar resultados
metrics_df = pd.DataFrame(columns=['max_depth', 'n_estimators', 'R2', 'MSE', 'RMSE', 'MAE', 'MAPE'])

# Usar 'itertools.product' para combinar 'max_depth' e 'n_estimators'
for m, n in product(max_depth_array, n_estimators_list):
    # Definir e treinar o modelo RandomForestRegressor
    model = RandomForestRegressor(max_depth=m, n_estimators=n, random_state=0)
    model.fit(X_train, y_train)

    # Predição sobre os dados de validação
    y_pred = model.predict(X_val)
    
    # Calcular métricas de performance
    r2_tuning  = r2_score( y_val, y_pred )
    mse_tuning  = mean_squared_error( y_val, y_pred )
    rmse_tuning = root_mean_squared_error( y_val, y_pred )
    mae_tuning  = mean_absolute_error( y_val, y_pred )
    mape_tuning = mean_absolute_percentage_error( y_val, y_pred )
    
    # Armazenar os resultados em uma nova linha como DataFrame
    new_row = pd.DataFrame({
        'max_depth': [m],
        'n_estimators': [n],
        'R2': [r2_tuning],
        'MSE': [mse_tuning],
        'RMSE': [rmse_tuning],
        'MAE': [mae_tuning],
        'MAPE': [mape_tuning]
    })

    # Concatenar a nova linha ao DataFrame existente
    metrics_df = pd.concat([metrics_df, new_row], ignore_index=True)

In [9]:
# Valores das métricas de performance para diferentes combinações dos hiperparâmetros 
metrics_df

,max_depth,n_estimators,R2,MSE,RMSE,MAE,MAPE
0,2,100,0.046667,455.227690,21.336065,16.905329,8.492380
1,2,500,0.046521,455.297136,21.337693,16.910023,8.488996
2,2,1000,0.046259,455.422186,21.340623,16.913442,8.486799
3,3,100,0.067885,445.095592,21.097289,16.760751,8.478494
4,3,500,0.067630,445.217333,21.100174,16.762433,8.473763
5,3,1000,0.067316,445.367331,21.103728,16.764876,8.472007
6,4,100,0.084225,437.293233,20.911557,16.633462,8.455013
7,4,500,0.084962,436.941276,20.903140,16.626421,8.457144
8,4,1000,0.084400,437.209511,20.909555,16.631766,8.458157
9,5,100,0.099717,429.895382,20.733919,16.495320,8.394029


Em particular, estamos interessados em buscar o conjunto de hiperparâmetros que minimize a **Raiz do Erro Quadrático Médio (RMSE)**. 

In [10]:
# Exibe as 5 primeiras linhas do dataset ordenado do menor para o maior RMSE
metrics_df.sort_values(['RMSE'], ascending=True)[:5]

,max_depth,n_estimators,R2,MSE,RMSE,MAE,MAPE
52,19,500,0.334069,317.989612,17.832263,13.204940,7.096579
53,19,1000,0.333654,318.187732,17.837817,13.206399,7.108343
49,18,500,0.330600,319.646074,17.878649,13.305202,7.134584
50,18,1000,0.330328,319.776153,17.882286,13.309967,7.146234
51,19,100,0.329769,320.043037,17.889747,13.262631,7.089372


A partir da primeira linha do dataset **ordenado** acima, é possível concluir que os melhores valores dos hiperparâmetros são: `max_depth` = 19 e `n_estimators` = 500. 

In [7]:
best_max_depth = 19
best_n_estimators = 500

## 5.4 - Desempenho do modelo para os dados de teste

In [8]:
# Definição do modelo
random_forest_test = RandomForestRegressor(
                                           max_depth = best_max_depth, 
                                           n_estimators = best_n_estimators, 
                                           random_state = 0
)

# Juntar os dados de treino e validação
random_forest_test.fit( pd.concat( [X_train, X_val] ),
                        pd.concat( [y_train, y_val] ) )

# Predição sobre os dados de teste
y_pred_test = random_forest_test.predict( X_test )

# Métricas
df_metrics_test = metrics(y_test, y_pred_test, 'Test', 'Random Forest Regressor')
df_metrics_test

,Algorithm,Step,R2,MSE,RMSE,MAE,MAPE
0,Random Forest Regressor,Test,0.3972,293.492,17.1316,12.638,6.3788


# 6.0 - Gera as planilhas com os resultados

In [9]:
exporta_excel('Random_Forest')